<a href="https://colab.research.google.com/github/TonyIssacJames/eva04_mar_2020_public/blob/master/EVA4_Assignment_04_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.8775, Accuracy: 2871/10000 (29%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.6608, Accuracy: 3926/10000 (39%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.4987, Accuracy: 8903/10000 (89%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2633, Accuracy: 8911/10000 (89%)



loss=0.12536147236824036 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.52it/s]



Test set: Average loss: 0.2602, Accuracy: 8918/10000 (89%)



## Let's try our model now

In [7]:
class Net_Mine(nn.Module):
    def __init__(self):
        super(Net_Mine, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.bn1   = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 28, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(28)
        self.pool1 = nn.MaxPool2d(2, 2)#14x14
        self.conv3 = nn.Conv2d(28, 20, 3, padding=1)
        self.bn3   = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20, 27, 3, padding=1)
        self.bn4   = nn.BatchNorm2d(27)
        self.pool2 = nn.MaxPool2d(2, 2) #7x7
        self.conv5 = nn.Conv2d(27, 16, 3)
        self.bn5   = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 10, 3)
        self.bn6   = nn.BatchNorm2d(10)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        self.avg_pool = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x))))))
        #x = F.relu(self.avg_pool(x))
        x = self.avg_pool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

model_mine = Net_Mine().to(device)
summary(model_mine, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 28, 28, 28]           4,060
       BatchNorm2d-4           [-1, 28, 28, 28]              56
         MaxPool2d-5           [-1, 28, 14, 14]               0
            Conv2d-6           [-1, 20, 14, 14]           5,060
       BatchNorm2d-7           [-1, 20, 14, 14]              40
            Conv2d-8           [-1, 27, 14, 14]           4,887
       BatchNorm2d-9           [-1, 27, 14, 14]              54
        MaxPool2d-10             [-1, 27, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           3,904
      BatchNorm2d-12             [-1, 16, 5, 5]              32
           Conv2d-13             [-1, 10, 3, 3]           1,450
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:

model_mine = Net_Mine().to(device)
optimizer  = optim.SGD(model_mine.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model_mine, device, train_loader, optimizer, epoch)
    test(model_mine, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0796, Accuracy: 9874/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0577, Accuracy: 9888/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0255, Accuracy: 9924/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0224, Accuracy: 9925/10000 (99%)



loss=0.009081949479877949 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 43.67it/s]



Test set: Average loss: 0.0227, Accuracy: 9934/10000 (99%)



#### We have achieved the test accuracy 9944/10000 in the 15 the epoch!, and met the target


## Model with Dropout

In [0]:
import torch.nn.functional as F

In [10]:
class Net_Mine(nn.Module):
    def __init__(self):
        super(Net_Mine, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), #input -? OUtput? RF
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 28, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(28),
            nn.MaxPool2d(2, 2),#14x14
            nn.Dropout(0.25)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(28, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 27, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(27),
            nn.MaxPool2d(2, 2), #7x7
            nn.Dropout(0.25)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(27, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.20)
        )

        self.avg_pool = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #x = F.relu(self.avg_pool(x))
        x = self.avg_pool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

model_mine = Net_Mine().to(device)
summary(model_mine, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 28, 28, 28]           4,060
              ReLU-5           [-1, 28, 28, 28]               0
       BatchNorm2d-6           [-1, 28, 28, 28]              56
         MaxPool2d-7           [-1, 28, 14, 14]               0
           Dropout-8           [-1, 28, 14, 14]               0
            Conv2d-9           [-1, 20, 14, 14]           5,060
             ReLU-10           [-1, 20, 14, 14]               0
      BatchNorm2d-11           [-1, 20, 14, 14]              40
           Conv2d-12           [-1, 27, 14, 14]           4,887
             ReLU-13           [-1, 27, 14, 14]               0
      BatchNorm2d-14           [-1, 27,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:
model_mine = Net_Mine().to(device)
optimizer  = optim.SGD(model_mine.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model_mine, device, train_loader, optimizer, epoch)
    test(model_mine, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0669, Accuracy: 9850/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0383, Accuracy: 9900/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0195, Accuracy: 9941/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0165, Accuracy: 9948/10000 (99%)



loss=0.024112530052661896 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.38it/s]



Test set: Average loss: 0.0194, Accuracy: 9938/10000 (99%)



#### We have achieved the maximum test accuracy  9948/10000, and met the target 99.4
